<a href="https://colab.research.google.com/github/jkordonis/TropicalFactorization/blob/main/MovieLensTropFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import cupy as np
#import numpy as np


In [39]:
# @title Load the MovieLens data (run this cell).

# Download MovieLens data.
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')





Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [40]:
indices_pd = ratings[['user_id', 'movie_id']].values
IND=np.array(indices_pd).astype(int)
IND_x,IND_y = IND[:,0],IND[:,1]

n = 944#(np.max(IND,0)[0]+1).astype(int)
p = 1683# (np.max(IND,0)[1]+1).astype(int)
r = 30
B=np.zeros((n,p))+1


B[IND_x,IND_y]=-1

In [41]:
MEanMat=B*0-B.mean()

In [42]:
pairs=[(i,j) for j in range(p) for i in range(n)]
pairs=np.array(pairs)
np.random.shuffle(pairs)

N=1271000 #n*p*0.8
N_=1429876
N__=n*p


In [43]:
IND_x_train = pairs[:N,0]
IND_y_train = pairs[:N,1]


IND_x_val  = pairs[N:N_,0]
IND_y_val  = pairs[N:N_,1]

IND_x_tst  = pairs[N_:N__,0]
IND_y_tst  = pairs[N_:N__,1]





B_sparse_ob_or = -B[IND_x_train,IND_y_train]
B_sparse_ob_or_val = -B[IND_x_val,IND_y_val]
B_sparse_ob_or_tst = -B[IND_x_tst,IND_y_tst]


In [44]:
 # Initialize C,D matrix estimates
C,D_Tr=np.random.rand(n,r),np.random.rand(p,r)


In [45]:
import time
T=time.time()
cnt=0
Err =np.zeros(100000)
Err_val =np.zeros(100000)
Err_tst =np.zeros(100000)


for _ in range(100):
  for _ in range(1):
    INDICES=np.array(range(N))
    np.random.shuffle(INDICES)
    BatchSize=1000
    i=0
    minibatches=[]
    while i*BatchSize<=N:
      minibatches.append(INDICES[i*BatchSize:BatchSize*(i+1)])
      i+=1

    #minibatches = [INDICES[i*BatchSize:BatchSize*(i+1)] for i in range(np.ceil(N/BatchSize).astype(int))]
    for AA in minibatches:
      IND_x_ob=IND_x_train[AA]
      IND_y_ob=IND_y_train[AA]
      B_sparse_ob=B_sparse_ob_or[AA]

      Maximizers = np.argmax(C[IND_x_ob]+D_Tr[IND_y_ob],-1)
      B_sparse_ob_est = C[IND_x_ob,Maximizers]+D_Tr[IND_y_ob,Maximizers]
      Differences = B_sparse_ob_est -B_sparse_ob
    #  for i in range(IND_x_ob.shape[0]):
      C_to_change = C[IND_x_ob,Maximizers]
      D_to_change = D_Tr[IND_y_ob,Maximizers]

      C[IND_x_ob,Maximizers]=C_to_change-0.03*Differences
      D_Tr[IND_y_ob,Maximizers]=D_to_change-0.03*Differences


    Maximizers = np.argmax(C[IND_x_train]+D_Tr[IND_y_train],-1)
    B_sparse_ob_est = C[IND_x_train,Maximizers]+D_Tr[IND_y_train,Maximizers]
    B_sparse_ob=B_sparse_ob_or[range(N)]
    Differences = B_sparse_ob_est -B_sparse_ob
    Err[cnt]=np.linalg.norm(Differences)
  #  C=C+0.0001
  #  D_Tr=D_Tr+0.0001

    Maximizers = np.argmax(C[IND_x_tst]+D_Tr[IND_y_tst],-1)
    B_sparse_ob_est = C[IND_x_tst,Maximizers]+D_Tr[IND_y_tst,Maximizers]
    Differences_tst = B_sparse_ob_est -B_sparse_ob_or_tst

    Maximizers = np.argmax(C[IND_x_val]+D_Tr[IND_y_val],-1)
    B_sparse_ob_est = C[IND_x_val,Maximizers]+D_Tr[IND_y_val,Maximizers]
    Differences_val = B_sparse_ob_est -B_sparse_ob_or_val
    Err_val[cnt]=np.linalg.norm(Differences_val)
    Err_tst[cnt]=np.linalg.norm(Differences_tst)

    cnt+=1



  print(time.time()-T,Err[cnt-1],np.linalg.norm(Differences_val),np.linalg.norm(Differences_tst))

1.4867136478424072 582.8669669679366 206.3060355829864 206.0401198456392
2.9202792644500732 492.05022735099254 174.45370422528808 174.01065859381225
4.330198049545288 473.30863232722675 168.37626123446037 167.8261003866533
5.770934104919434 464.2689068618388 165.55122266480205 165.08392772793414
7.641326427459717 458.669434749528 164.01687069704855 163.65626766636979
9.514187335968018 453.7316325667895 162.56188008316337 161.99977235972298
10.93578553199768 450.49112652040037 161.67748785138158 161.36542366467742
12.34981894493103 447.0719330944579 160.7687462791619 160.2757878387907
13.740629434585571 444.5632007024541 160.13640573699854 159.63020436312806
15.146343469619751 442.249721420496 159.74029994924427 159.00814071432637
16.547549962997437 440.11673128037495 159.30903178414997 158.7244835653504
17.939523458480835 438.9269874503219 159.30092596556696 158.46632312448455
19.763822317123413 437.28565747182984 158.89134785483859 158.3025749765019
21.73359704017639 436.1829672397113

KeyboardInterrupt: ignored

In [ ]:
indexmin=np.argmin(    Err_val[:cnt])
print(Err_tst[indexmin])

In [ ]:
import matplotlib.pyplot as plt
import numpy

#plt.plot(np.asnumpy(Err[:cnt]))
plt.plot(np.asnumpy(Err_tst[:cnt]))
plt.plot(np.asnumpy(Err_val[:cnt]))
plt.grid()
#plt.plot(Err[:cnt])